<a href="https://colab.research.google.com/github/postak/colazione-con-adk/blob/main/2025_09_Partners_ADK_Learning_session_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```
Copyright 2025 Google LLC.
SPDX-License-Identifier: Apache-2.0
```

In [ ]:
#@title Last Session
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Advanced Agent Development: From Deployment to Evaluation on Vertex AI

Welcome! 🚀

This notebook is your comprehensive guide to the full lifecycle of building, deploying, and evaluating sophisticated AI agents using the Google Agent Development Kit (ADK) and Vertex AI.

**Our journey will follow a specific, educational flow:**
1.  **Agent #1 (Standard Deployment)**: We'll create a multi-agent trip planner with **custom Python tools** and deploy it *without* tracing to see a baseline deployment.
2.  **Understanding Sessions**: We'll query our first agent with and without sessions to clearly demonstrate the difference between stateless and stateful (short-term memory) conversations.
3.  **Agent #2 (Memory & Tracing)**: We'll build a second agent with a long-term **Memory Bank** and deploy it *with* **tracing** enabled.
4.  **Comparing Tracing**: We'll see the practical difference in observability between an agent deployed with and without tracing.
5.  **Comprehensive Evaluation**: Finally, we'll run a rigorous evaluation on our first agent to measure its performance, tool selection, and even the accuracy of the arguments passed to its tools.

Let's get started!


-------------
### 🎁 🛑 Important Prerequisite: Setup Your Environment! 🛑 🎁
-----------------------------------------------------------------------------

You will need a **Google AI API Key** to run this notebook.

👉 Follow the instructions [here](https://github.com/postak/colazione-con-adk/blob/main/Setting%20Up%20Your%20GCP%20Project%20%26%20Gemini%20API%20Key.pdf)

 -----------------------------------------------------------------------------

---

## 1. Setup and Configuration

First, we'll install the necessary packages and configure our Google Cloud environment.

### Install Libraries
This command installs the Google Cloud AI Platform SDK, including support for the Agent Development Kit (`adk`), evaluation, and Agent Engines.

In [ ]:
# --- 1. Installation ---
%pip install --upgrade --quiet 'google-cloud-aiplatform[evaluation,adk,agent_engines]'
%pip install --upgrade --quiet 'google-api-python-client'
%pip install --upgrade --quiet 'google-adk==1.15.1'
%pip install --upgrade --quiet 'google-cloud-aiplatform==1.117.0'

### Import Libraries
Now, let's import all the libraries we'll need for this session.

In [ ]:
# --- 2. Import all necessary libraries ---
import os
import sys
import json
import asyncio
import random
import string
from uuid import uuid4
from typing import Any, List

import pandas as pd
import plotly.graph_objects as go
import vertexai
from google.colab import auth
from IPython.display import HTML, Markdown, display

# --- ADK, Agent, and Evaluation Components ---
from google.adk.agents import Agent
from google.adk.events import Event
from google.adk.runners import Runner
import google.adk as adk
from google.adk.memory import VertexAiMemoryBankService
from google.adk.sessions import VertexAiSessionService, InMemorySessionService
from google.genai import types
from vertexai import agent_engines
from vertexai.preview import reasoning_engines
from vertexai.preview.evaluation import EvalTask

print("✅ All libraries are ready to go!")

### Authenticate and Configure Your Project
To use Vertex AI, you need an active Google Cloud project. This section handles authenticating your environment and setting up the necessary project configurations.

In [ ]:
# --- 3. Authentication & Project Configuration ---

# Authenticate user in Colab
if "google.colab" in sys.modules:
    auth.authenticate_user()
    print("✅ Authenticated successfully.")

# @title Set Your Google Cloud Project Details
PROJECT_ID = ""  # @param {type:"string"}
LOCATION = ""          # @param {type:"string"}
STAGING_BUCKET = f"gs://{PROJECT_ID}-adk-staging" # A unique name for your bucket

# Set environment variables for the ADK and gcloud
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "True"

# Create the staging bucket if it doesn't exist
!gcloud storage buckets create {STAGING_BUCKET} --project={PROJECT_ID} --location={LOCATION} -l {LOCATION} 2>/dev/null

# Initialize the Vertex AI SDK
# We'll also create an "Experiment" to group our evaluation runs
EVAL_EXPERIMENT_NAME = f"trip-planner-evaluation-{uuid4().hex[:6]}"
vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET, experiment=EVAL_EXPERIMENT_NAME)

print(f"\n✅ Vertex AI configured for project '{PROJECT_ID}' in '{LOCATION}'.")
print(f"🔬 Evaluation results will be saved to experiment: {EVAL_EXPERIMENT_NAME}")

---

## 2. Agent #1: The Trip Planner (Standard Deployment)

We'll build our first agent, a "team" of specialists. Crucially, we will now provide them with our own **custom Python functions as tools** instead of a generic search tool. This makes their behavior deterministic and much easier to evaluate accurately.

### Define Custom Tools

These functions act as our mock APIs or databases. The agent will learn to call these with the correct parameters based on the user's request.

In [ ]:
# --- Custom Tool Definitions ---

def get_travel_attractions(location: str, interest: str = "any"):
    """
    Finds travel attractions in a given location based on an interest.

    Args:
        location: The city or place to search for attractions (e.g., "Paris", "Rome").
        interest: The type of attraction to look for (e.g., "museum", "historical site", "park").
    """
    # This is a mock database for demonstration
    attractions = {
        "paris": {"museum": "The Louvre", "historical site": "Eiffel Tower"},
        "rome": {"historical site": "The Colosseum", "museum": "Vatican Museums"},
        "tokyo": {"museum": "teamLab Borderless", "park": "Shinjuku Gyoen"},
        "san francisco": {"historical site": "Alcatraz Island", "park": "Golden Gate Park"}
    }

    city = attractions.get(location.lower(), {})
    if interest == "any":
        return city or f"Sorry, I don't have any information for {location}."
    else:
        return city.get(interest.lower(), f"Sorry, I couldn't find a {interest} in {location}.")

def get_restaurant_recommendations(location: str, cuisine: str):
    """
    Recommends a restaurant in a location based on the desired cuisine.

    Args:
        location: The city where the user wants to eat (e.g., "Paris", "Rome").
        cuisine: The type of food they want (e.g., "pasta", "seafood", "cafe").
    """
    # This is a mock database for demonstration
    restaurants = {
        "rome": {"pasta": "Trattoria Da Enzo al 29"},
        "san francisco": {"seafood": "Hog Island Oyster Co."},
        "paris": {"cafe": "Les Deux Magots", "fancy": "Le Cinq"}
    }

    city = restaurants.get(location.lower(), {})
    return city.get(cuisine.lower(), f"Sorry, I couldn't find a {cuisine} restaurant in {location}.")

### Define and Deploy the Agent Team (Without Tracing)
Now, we'll update our specialist agents to use these new, more specific tools.

In [ ]:
# --- Specialist Agent Definitions using Custom Tools ---
def create_activities_agent():
    return Agent(
        name="activities_agent", model="gemini-2.5-pro",
        description="Specialist for finding interesting activities, attractions, and historical sites.",
        instruction="",
        tools=[get_travel_attractions],
    )

def create_food_agent():
    return Agent(
        name="food_agent", model="gemini-2.5-pro",
        description="Specialist for finding great restaurants based on cuisine.",
        instruction="",
        tools=[get_restaurant_recommendations],
    )

# --- Coordinator Agent Definition ---
def create_trip_coordinator_agent(sub_agents: List[Agent]):
    return Agent(
        name="trip_coordinator_agent", model="gemini-2.5-pro",
        description="The master coordinator for trip planning. It delegates tasks to specialist agents.",
        sub_agents=sub_agents,
        instruction="""
        You are the master "Trip Coordinator" 🧭. Your job is to plan a full itinerary by delegating to your specialist agents.
        Extract the location, interests, and cuisine types from the user's prompt to use as arguments for your specialist agents.
        Once you have the information, combine the it  into a single, cohesive itinerary and present it in a clean MARKDOWN format.
        """,
    )


In [ ]:
# --- Assemble and Deploy the Team ---
trip_coordinator_agent = create_trip_coordinator_agent(
    sub_agents=[create_activities_agent(), create_food_agent()]
)
print(f"👑 Agent '{trip_coordinator_agent.name}' is ready for deployment!")

deployment_name = f"trip-planner-basic-{uuid4().hex[:6]}"
print(f"🚀 Deploying agent as '{deployment_name}' without tracing...")

deployed_agent_no_trace = agent_engines.create(
    display_name=deployment_name,
    agent_engine=trip_coordinator_agent, # Pass the agent object directly
    requirements=["google-cloud-aiplatform[agent_engines]==1.117.0", "google-adk==1.15.1"],
    description="A multi-agent trip planning system with custom tools and tracing disabled."
)

print("-" * 50)
print("✅ Agent System deployed successfully!")
display(Markdown(f"**Resource Name:** `{deployed_agent_no_trace.resource_name}`"))

---

## 3. Interacting with Agent #1: Understanding Sessions

Now we'll query our deployed agent. The goal here is to understand the crucial difference between a single, stateless query and a stateful conversation that uses a session for short-term memory.

### Helper Function for Clean Interaction
To avoid repeating code, we'll create one reusable helper function.

In [ ]:
async def stream_and_print_response(agent, message: str, user_id, session) -> str:

  events = []
  async for event in agent.async_stream_query(
      user_id=user_id,
      session_id = session["id"],
      message=message,
  ):
    events.append(event)

  print("--- Full Event Stream ---")
  for event in events:
      print(event)

  # For quick tests, you can extract just the final text response
  final_text_responses = [
      e for e in events
      if e.get("content", {}).get("parts", [{}])[0].get("text")
      and e.get("finish_reason", {})
      # and not e.get("content", {}).get("parts", [{}])[0].get("function_call")
  ]
  if final_text_responses:
      print("\n--- Final Response ---")
      output = ""
      for final_text_response in final_text_responses:
        print(final_text_response["content"]["parts"][0]["text"])
        output += final_text_response["content"]["parts"][0]["text"]
      return (output)
  print ("No response from agent")


In [ ]:
# !!!!! To speed up, let use an agent already deployed
# deployed_agent_no_trace = agent_engines.get("tbd")


### Querying With a Session
we'll show the behaviour of the session to enable short-term memory for a single, continuous conversation.

In [ ]:
print("\n\n🗣️ Querying WITH a session (Stateful Conversation)")

user_id = f"session-demo-user-{uuid4().hex[:6]}"

session = await deployed_agent_no_trace.async_create_session(user_id=user_id)
print (session)

# --- First Query ---
query_1 = "My name is Annie. I want to plan a trip to Rome."
print(f"\n💬 User: {query_1}")
await stream_and_print_response(deployed_agent_no_trace, query_1, user_id, session)

print("\n" + "-" * 50)


# --- Follow-up Query (in the same session) ---
query_2 = "Find a famous historical site there and a good place for pasta."
print(f"\n💬 User: {query_2}")
await stream_and_print_response(deployed_agent_no_trace, query_2, user_id, session)

print("\n" + "-" * 50)

# --- Follow-up Query (in the same session) ---
query_3 = "What is my name?"
print(f"\n💬 User: {query_3}")
await stream_and_print_response(deployed_agent_no_trace, query_3,user_id, session)

print("\n" + "-" * 50)

# --- Follow-up Query (in the different session) ---
session2 = await deployed_agent_no_trace.async_create_session(user_id=user_id)
print (session2)
print(f"\n💬 User: {query_3} Now in different session")
await stream_and_print_response(deployed_agent_no_trace, query_3,user_id, session2)


print("\n" + "-" * 50)
# print("Success! The agent remembered the location ('Rome') from the first message to correctly answer the second.")

---

## 4. Agent #2: Adding Long-Term Memory & Tracing
Session memory is temporary. For an agent to remember preferences across *different conversations*, it needs a **Memory Bank**.

Let's build a new agent designed for this. This time, we **will enable tracing** so we can see its inner workings.

### Define and Deploy the Memory Agent (With Tracing)

In [ ]:
from google.genai.types import Content, Part


def create_memory_trip_agent():
    """Creates a trip planner that uses a long-term memory bank."""
    return Agent(
        name="memory_trip_agent", model="gemini-2.5-flash",
        instruction="""You are a helpful trip planning assistant with a perfect memory.
        Your Core Instructions:
        1. At the start of every conversation, use your memory to recall any user preferences.
        2. When the user tells you a new preference (e.g., "I'm a vegetarian"), save this fact to your memory.
        3. When planning, always incorporate the user's remembered preferences.
        """,
        tools=[adk.tools.preload_memory_tool.PreloadMemoryTool()]
    )

memory_agent = create_memory_trip_agent()
print("🧠 Agent with long-term memory is defined.")

In [ ]:
# --- 9. Configure and Run the Local Agent with Memory Bank ---
async def run_memory_test():
    # We will attach our new memory bank to the Agent Engine we already deployed.
    agent_engine_id = deployed_agent_no_trace.name
    app_name = memory_agent.name # Use the agent's name as the app identifier

    print(f"✅ Attaching Memory & Session services to Agent Engine: {agent_engine_id}")

    # Configure services to point to our existing deployment
    memory_bank_service = VertexAiMemoryBankService(
        project=PROJECT_ID, location=LOCATION, agent_engine_id=agent_engine_id
    )
    session_service = VertexAiSessionService(
        project=PROJECT_ID, location=LOCATION, agent_engine_id=agent_engine_id
    )

    # Initialize the local runner once
    runner = Runner(
        agent=memory_agent,
        app_name=app_name,
        session_service=session_service,
        memory_service=memory_bank_service,
    )

    # ---
    # Helper function to run a single turn in a chat
    async def run_turn(query: str, user_id: str, session_id: str):
        print(f"\n🗣️ You: {query}")
        print("🤖 Assistant: ", end="", flush=True) # Print prefix for streaming

        final_response_parts = []
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session_id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            # ⭐️ FIX: Access event data using dot notation for objects.
            if event.content and event.content.role == "model":
                part_text = event.content.parts[0].text
                if part_text:
                    final_response_parts.append(part_text)
                    print(part_text, end="", flush=True) # Print streaming parts on one line

        print() # Add a final newline after the response is complete
        return "".join(final_response_parts)
    # ---

    # == Step 1: The "Teach Me" Conversation ==
    user_id_memory = f"adk-traveler-{uuid4()}"
    print(f"\n--- Starting Session 1 for user '{user_id_memory}' ---")
    session_one = await session_service.create_session(user_id=user_id_memory, app_name=app_name)
    await run_turn("Hi there. When you help me plan trips, please remember that I am a vegetarian.", user_id_memory, session_one.id)

    # ⭐️ IMPORTANT: Explicitly save the completed session to long-term memory
    print("\n💾 Saving session 1 to Memory Bank...")
    completed_session = await session_service.get_session(app_name=app_name, user_id=user_id_memory, session_id=session_one.id)
    await memory_bank_service.add_session_to_memory(completed_session)
    print("✅ Session 1 saved.")

    # == Step 2: The "Recall" Conversation ==
    print(f"\n--- Starting a brand new Session 2 for the SAME user ---")
    session_two = await session_service.create_session(user_id=user_id_memory, app_name=app_name)
    await run_turn("Great. Now, can you plan a weekend dinner trip to Napa Valley?", user_id_memory, session_two.id)
    print("-" * 50)
    print("🎉 Memory test complete!")

# In a notebook cell, run the test with await
await run_memory_test()

### Tracing

In [ ]:
# This time, we wrap the agent in AdkApp to enable tracing

from vertexai.agent_engines import AdkApp

app_with_tracing = AdkApp(agent=trip_coordinator_agent, enable_tracing=True)
deployment_name = f"trip-planner-tracing-{uuid4().hex[:6]}"

print(f"🚀 Deploying '{deployment_name}' with tracing enabled...")
deployed_agent = agent_engines.create(
    display_name=deployment_name,
    agent_engine=app_with_tracing,
    requirements=["google-cloud-aiplatform[agent_engines]==1.117.0", "google-adk==1.15.1"],
    description="A trip planning agent with  tracing enabled."
)
print("-" * 50)
print("✅ Agent with Tracing deployed successfully!")
display(Markdown(f"**Resource Name:** `{deployed_agent.resource_name}`"))

---

## 5. Comparing Agents in Cloud Trace
Now, let's see the difference tracing makes.

1.  Navigate to the **[Cloud Trace](https://console.cloud.google.com/traces)** page in your Google Cloud Console.
2.  In the filter or search bar, look for traces related to your Reasoning Engines. You can often filter by the service name, which will contain your deployment name.

You will find detailed, step-by-step execution graphs for **`trip-planner-trace-...`** (Agent #2). However, you will find **no detailed traces** for **`trip-planner-basic-...`** (Agent #1), because we deployed it without enabling tracing. This demonstrates the critical importance of the `AdkApp` wrapper for observability and debugging.

### Query The Remote Agent
We'll have one conversation to teach the agent a preference, and then start a completely **new session** to see if it remembers.

In [ ]:
# deployed_agent = agent_engines.get("projects/418653888075/locations/us-central1/reasoningEngines/2984290062063108096")


In [ ]:
# agent_client = agent_engines.get(deployed_agent.resource_name)
user_id_memory = f"trace-test-user-{uuid4().hex[:6]}"

# --- Conversation 1: Teach the Agent a Fact ---
print(f"--- 📚 Starting Conversation 1 for user '{user_id_memory}' ---")
query1 = "Find a famous historical site there and a good place for pasta in Rome."
print(f"💬 User: {query1}")
session1 = await deployed_agent.async_create_session(user_id=user_id_memory)
await stream_and_print_response(deployed_agent,query1, user_id_memory, session1)


print("\n" + "-" * 50)
print("✅ Trace test complete.")

---

## 6. Cleanup

Finally, let's clean up the resources we've created to avoid incurring unnecessary costs. This will delete the deployed Agent Engines and the Vertex AI Experiment.

In [ ]:
# --- Delete the deployed Agent Engines ---
all_deployments = [deployed_agent_no_trace, deployed_agent]
for deployment in all_deployments:
    try:
        deployment.delete()
        print(f"🗑️ Deleted Agent Engine: {deployment.display_name}")
    except Exception as e:
        print(f"Could not delete {deployment.display_name}. It may have been deleted already. Error: {e}")